<a href="https://colab.research.google.com/github/redish0317/cp1/blob/main/cp1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import time
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

def crawl_id(limit=100,offset=0):
    url = 'http://www.wanted.jobs/api/v4/jobs?'

    params ={1656232918453:'',    #사용자번호?
    'country': 'all',
    'tag_type_ids': 873,    #직무 카테고리 고유 id
    'job_sort': 'job.latest_order',    #최신순 정렬
    'locations': all,
    'years': -1,    #경력 이상
    'years': -1,    #경력 이하    경력상관없이 검색하려면 -1
    'limit': limit,    #한 번에 조회 가능한 수 (최대100)
    'offset': offset}    #조회할 게시물의 첫 index        ex) limit=100 offset=10  => 10번게시물부터 110번게시물까지 크롤링

    #서버에 url과 쿼리로 요청
    r = requests.get(url,
                     params = params)
    #요청한 데이터 json포멧으로 변환
    r = r.json()
    #json포멧 데이터중 id컬럼만 추출
    id_list = [i['id'] for i in r['data']]
    return id_list

def return_id_list():
    '''
    0번째 게시물부터 100개씩 크롤링 while true
    오류발생! => ex) 총 게시물이 321개인데 300개 크롤링 후 다음100개를 크롤링하려했기때문
    따라서 재귀호출을 통해 크롤링 수를 100개씩 -> 오류발생! -> 10개씩 -> 오류발생! -> 1개씩 크롤링하는 함수구현
    만약 게시물이 321개라면 300개 크롤링 -> 20개 크롤링 - 1개 크롤링 return
    '''
    id_list=[]
    def crawl_all_id(limit=100,offset=0):
        try:
            while True:
                id_list.extend(crawl_id(limit,offset))
                offset+=limit

        except:
            if limit != 1:
                return crawl_all_id(limit/10,offset)
    crawl_all_id()
    return id_list


def crawl_job(id_list):
    df_list = []
    
    for id in id_list:
        url = f'https://www.wanted.jobs/api/v4/jobs/{id}?1656259528432'
        r = requests.get(url)
        r = r.json()['job']

        #1개의 게시물 크롤링할때마다 데이터프레임에 append 또는 concat하는것보다
        #list에 append하고 마지막에 한번에 concat하는게 속도가 더 빠르다고 함
        df_list.append(pd.json_normalize(r))
        
    df = pd.concat(df_list, ignore_index=True)
    #단일 게시물을 크롤링해서 concat하다보니 index가 모두 0이므로 reset_index
    return df


def engineering(df):
    drop_col = []
    df = df.drop(drop_col,axis=1)
    return df
start = time.time()

id_list = return_id_list()
end=time.time()
print(end-start)

df_job = crawl_job(id_list)
end=time.time()
print(end-start)

89.12712955474854
1693.5257630348206


In [26]:
df_job

,is_crossboarder,like_count,id,is_like,due_time,score,company_images,skill_tags,status,is_bookmark,...,title_img.origin,title_img.thumb,reward.formatted_total,reward.formatted_recommender,reward.formatted_recommendee,address.geo_location.bounds.northeast.lat,address.geo_location.bounds.northeast.lng,address.geo_location.bounds.southwest.lat,address.geo_location.bounds.southwest.lng,address.geo_location
0,True,0,119349,False,2022-07-29,None,[{'url': 'https://static.wanted.co.kr/images/c...,"[{'title': 'MongoDB', 'id': 1462, 'kind_title'...",active,False,...,https://static.wanted.co.kr/images/company/571...,https://static.wanted.co.kr/images/company/571...,"￦1,000,000","￦500,000","￦500,000",NaN,NaN,NaN,NaN,NaN
1,True,16,22571,False,None,None,[{'url': 'https://static.wanted.co.kr/images/c...,"[{'title': 'Android', 'id': 1438, 'kind_title'...",active,False,...,https://static.wanted.co.kr/images/company/539...,https://static.wanted.co.kr/images/company/539...,"￦1,000,000","￦500,000","￦500,000",NaN,NaN,NaN,NaN,NaN
2,True,43,21449,False,None,None,[{'url': 'https://static.wanted.co.kr/images/c...,"[{'title': 'Android', 'id': 1438, 'kind_title'...",active,False,...,https://static.wanted.co.kr/images/company/539...,https://static.wanted.co.kr/images/company/539...,"￦1,000,000","￦500,000","￦500,000",NaN,NaN,NaN,NaN,NaN
3,True,217,45650,False,None,None,[{'url': 'https://static.wanted.co.kr/images/c...,"[{'title': 'iOS', 'id': 1457, 'kind_title': 'S...",active,False,...,https://static.wanted.co.kr/images/company/153...,https://static.wanted.co.kr/images/company/153...,"￦1,000,000","￦500,000","￦500,000",NaN,NaN,NaN,NaN,NaN
4,True,0,119342,False,None,None,[{'url': 'https://static.wanted.co.kr/images/c...,"[{'title': 'Git', 'id': 1411, 'kind_title': 'S...",active,False,...,https://static.wanted.co.kr/images/company/274...,https://static.wanted.co.kr/images/company/274...,"￦1,000,000","￦500,000","￦500,000",NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2072,True,13,24751,False,None,None,[{'url': 'https://static.wanted.co.kr/images/c...,[],active,False,...,https://static.wanted.co.kr/images/company/744...,https://static.wanted.co.kr/images/company/744...,,,,35.673279,139.723609,35.672718,139.722551,NaN
2073,True,30,14696,False,None,None,[{'url': 'https://static.wanted.co.kr/images/c...,[],active,False,...,https://static.wanted.co.kr/images/company/387...,https://static.wanted.co.kr/images/company/387...,"NT$15,000","NT$10,000","NT$5,000",NaN,NaN,NaN,NaN,NaN
2074,True,77,9503,False,None,None,[{'url': 'https://static.wanted.co.kr/images/c...,[],active,False,...,https://static.wanted.co.kr/images/company/273...,https://static.wanted.co.kr/images/company/273...,"SG$1,700","SG$1,000",SG$700,1.285252,103.850611,1.281158,103.849886,NaN
2075,True,177,9020,False,None,None,[{'url': 'https://static.wanted.co.kr/images/c...,[],active,False,...,https://static.wanted.co.kr/images/company/261...,https://static.wanted.co.kr/images/company/261...,"SG$1,700","SG$1,000",SG$700,1.293777,103.851951,1.254780,103.796450,NaN


In [28]:
df_job.to_csv('wanted.csv')